In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Dropout, LayerNormalization, MultiHeadAttention, Add
from tensorflow.keras.layers import Input, Embedding, Concatenate
from tensorflow.keras.models import Model

In [ ]:
""" Change the config value to build
different ViT variant """

config = {}
config["num_layers"] = 12
config["hidden_dim"] = 768
config["mlp_dim"] = 3072
config["num_heads"] = 12
config["dropout_rate"] = 0.1
config["num_patches"] = 256
config["patch_size"] = 32
config["num_channels"] = 3



""" Model Hyperparameters """
hp = {}

hp["num_classes"] = 5

In [ ]:

class ClassToken(Layer):
  def __init__(self):
    super().__init__()

  def build(self, input_shape):
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(
        initial_value = w_init(shape=(1,1,input_shape[-1]), dtype=tf.float32),
        trainable=True
    )

  def call(self, inputs):
    batch_size = tf.shape(inputs)[0]
    hidden_dim = self.w.shape[-1]

    cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
    # Broadcasting is the process of making arrays to have compatible shapes for arithmetic operations.
    #Two shapes are compatible if for each dimension pair they are either equal or one of them is one.

    cls = tf.cast(cls, dtype = inputs.dtype)
    return cls


def mlp(x,cf):
  x = Dense(cf["mlp_dim"], activation = "gelu")(x)
  x = Dropout(cf["dropout_rate"])(x)
  x = Dense(cf["hidden_dim"])(x)
  x = Dropout(cf["dropout_rate"])(x)
  return x


def transformer_encoder(x,cf):

  residual_1 = x

  x = LayerNormalization()(x)
  x = MultiHeadAttention(num_heads = cf["num_heads"], key_dim = cf["hidden_dim"])(x,x)

  x = Add()([x,residual_1])

  residual_2 = x

  x = LayerNormalization()(x)
  x = mlp(x,cf)

  x = Add()([x,residual_2])

  return x




def ViT(cf):

  """ Transformer Input """

  input_shape = (cf["num_patches"], cf["patch_size"]*cf["patch_size"]*cf["num_channels"])
  inputs = Input(input_shape) #(None, 256, 3072)
  print(inputs.shape)

  """ Patch + Position Embeddings """
  patch_embed = Dense(cf["hidden_dim"])(inputs)

  positions = tf.range(start=0, limit=cf["num_patches"], delta=1) ## (256,)
  pos_embed = Embedding(input_dim=cf["num_patches"], output_dim=cf["hidden_dim"])(positions) ## (256, 768)
  embed = patch_embed + pos_embed ## (None, 256, 768)



  """ Adding Class Token """
  token = ClassToken()(embed)

  x = Concatenate(axis = 1)([token,embed]) #(None, 257, 768) added one extra embeded learnable patch

  """ Transformer Encoder """

  for _ in range (cf["num_layers"]):
    x = transformer_encoder(x,cf)
    #print(x.shape)

  """ Classification Head """

  x = LayerNormalization()(x) ## (None, 257, 768)
  x = x[:, 0, :] ## (None, 768)
  x = Dropout(0.1)(x)
  x = Dense(hp["num_classes"], activation="softmax")(x)

  model = Model(inputs, x)
  return model






In [ ]:
if __name__ == "__main__":
  model = ViT(config)
  model.summary()

(None, 256, 3072)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 3072)]  0           []                               
                                                                                                  
 dense (Dense)                  (None, 256, 768)     2360064     ['input_1[0][0]']                
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 256, 768)    0           ['dense[0][0]']                  
 da)                                                                                              
                                                                                                  
 class_token (ClassToken)       (None, 1, 768)       768         ['tf.__oper